# Lecture 4: Numerics for Conservation Law

$$
\begin{aligned}
&u_t + f (u)_x = 0, \quad[\text{differential form}]\\
&u(x,0)=u^0(x).
\end{aligned}
$$

The concept of weak solution can be introduced by considering the integral form:
$$
\frac{d}{dt}
\int^b_a  u(x, t) dx=f(u(a, t))-f(u(b,t)) \quad[\text{intergral form}]
$$

For $C^1$ solutions, the solution defined by strong form and weak form are equivalent. Attempts at defining weak solutions:

- If u satisfies [intergral form] for almost all $(a,b)$ then in $u$ is called a weak solution to the differential form. (physically meaningful,correct)
- If for any $\phi\in C^1_0(\mathbb{R}^2)$,
$$ - \int_0^\infty\int_0^\infty (u\phi_t+f(u)\phi_x) dxdt- \int_{-\infty}^\infty u^0(x)\phi(x,0)dx=0,$$
&emsp;&emsp;then $u$ is called a weak solution to [differential form]. (more meaningful mathematically motivated by multiplication by test function and integration by parts.)

# The importance of conservation form

For a nonlinear problem like Burgers' equation, it is important to use a finite volume method that is in conservation form, or it is possible to converge to a function containing a shock wave that is not a solution of intended conservation law.

This illustrates Figure 12.5 in [FVMHP](http://depts.washington.edu/clawpack/book.html).

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function

In [3]:
import animation_tools  # local version, rather than from Clawpack

In [4]:
animation_style = 'ipywidgets'
#animation_style = 'JSAnimation'

## Desired true solution

Define the true solution as a function of $(x,t)$. For this example we have a Riemann problem with a shock wave solution.

Evaluating this function at $t=0$ will give the initial conditions below.

In [5]:
f = lambda q: 0.5*q**2

q_left = 3.
q_right = 1.
s = 0.5*(q_left + q_right)

xlower = -1.
xupper = 3.

def qtrue(x,t):
    # Riemann problem, assuming shock wave solution:
    x0 = x - s*t   # trace back characteristic to time 0
    q = where(x0<0, q_left, q_right) 
    return q

### Define upwind method with both conservative and non-conservative variants

Consider Burgers' Equation $q_t + \left(\frac 1 2 q^2\right)_x = 0$.

We assume $Q_i^n > 0$ everywhere so the upwind direction is to the left.

If `conservative = True` in the call to `upwind`, the conservative upwind method is used, differencing the flux function $f(q) = \frac 1 2 q^2$:
$$
Q_i^{n+1} = Q_i^n - \frac{\Delta t}{\Delta x} \left( \frac 1 2 (Q_i^n)^2 - \frac 1 2 (Q_{i-1}^n)^2 \right)
$$

If `conservative = False` in the call to `upwind`, the non-conservative upwind method is used, based on the quasi-linear form of Burgers' equation $q_t + qq_x = 0$:

$$
Q_i^{n+1} = Q_i^n - \frac{\Delta t}{\Delta x} Q_i^n \left( Q_i^n - Q_{i-1}^n \right)
$$

This function also make a plot every `nplot` time steps and accumulate these in `figs`, which is returned from the function call.


In [6]:
def upwind(x,tfinal,nsteps,dt,qtrue,nplot,conservative):
    
    dx = x[1] - x[0]  # assumes uniform grid
    
    # set initial data:
    q0 = qtrue(x,0.)

    # plot initial data:
    fig = figure()
    xfine = linspace(xlower,xupper,5000)   # fine grid for plotting true solution
    plot(xfine, qtrue(xfine,0.),'r-')
    title('Initial data')
    plot(x,q0,'bo')
    ylim(0,4)

    # Start accumulating figures
    # Will add more figures during time stepping
    figs = [fig]  
    close(fig)  # so it won't appear yet


    # extend to include 2 ghost cells on each side:
    qn_ext = hstack([0., 0., q0, 0., 0.])
    i1 = 2  # index of first interior cell
    imx = mx+1  # index of last interior cell

    for n in range(1, nsteps+1):

        # fill ghost cells for inflow BCs:
        qn_ext[i1-2] = q_left
        qn_ext[i1-1] = q_left
        #qn_ext[imx+1] = Not needed
        #qn_ext[imx+2] = 

        qnp_ext = qn_ext.copy()  # make a copy, not a new pointer to old array

        
        if conservative:
            for i in range(i1,imx+1):
                qnp_ext[i] = qn_ext[i] - dt/dx * (f(qn_ext[i])-f(qn_ext[i-1]))
                
        else:
            for i in range(i1,imx+1):
                qnp_ext[i] = qn_ext[i] - dt/dx * qn_ext[i]*(qn_ext[i]-qn_ext[i-1])

                
        qn_ext = qnp_ext

        if mod(n,nplot) == 0:
            fig = figure()
            qn = qn_ext[i1:imx+1]
            plot(x,qn,'bo-')
            ylim(0,4)
            tn = n*dt
            plot(xfine, qtrue(xfine,tn),'r-')
            title('Time t = %g' % tfinal)
            figs.append(fig)
            close(fig)

    print("Took %i time steps and produced %i figures" % (nsteps,len(figs)))
    return figs

In [7]:
mx = 100
dx = float(xupper-xlower)/mx

# finite volume cell centers:
x = linspace(xlower+0.5*dx, xupper-0.5*dx, mx)

tfinal = 1.0
nsteps = 100
dt = float(tfinal)/nsteps
cfl = max(q_left, q_right)*dt/dx
print("dx = %6.4f, dt = %6.4f, Courant number is cfl = %5.2f" % (dx,dt,cfl))

dx = 0.0400, dt = 0.0100, Courant number is cfl =  0.75


## Test the conservative upwind method:

In [8]:
figs = upwind(x,tfinal,nsteps,dt,qtrue,nplot=5,conservative=True)

anim = animation_tools.animate_figs(figs, style=animation_style, figsize=(10,5))
display(anim)

Took 100 time steps and produced 21 figures


interactive(children=(IntSlider(value=0, description='frameno', max=20), Output()), _dom_classes=('widget-inte…

None

In [9]:
figs = upwind(x,tfinal,nsteps,dt,qtrue,nplot=5,conservative=False)

anim = animation_tools.animate_figs(figs, style=animation_style, figsize=(10,5))
display(anim)

Took 100 time steps and produced 21 figures


interactive(children=(IntSlider(value=0, description='frameno', max=20), Output()), _dom_classes=('widget-inte…

None

**Definition** A scheme to solve conservation laws is called conservative if it can be written as
$$ u^{n+1}_j = u^n_j - \frac{\Delta t}{\Delta x} [\hat{f}_{j+1/2}-\hat{f}_{j-1/2}],$$
where $\hat{f}$ is 
1. Lipschitz continous
2. $\hat{f}(u, \cdots, u) = f(u)$ (consistency)

**Theorem** (**Lax-Wendroff**) If the solution $\{u^n_j\}$ to a conservative scheme converges (as $\Delta t$, $\Delta x\to 0$) boundedly a.e. to a function $u(x, t)$, then $u$ is a weak solution of the conservation law.

- Peter Lax, New York University, he thrived in the Courant Institute of Mathematical Sciences New York University where applied mathematics was studied alongside relevant pure mathematics in an exciting mix of ideas which led to great progress. 
- Burton Wendroff, Los Alamos National Laboratory, primary author of the Chess program Lachex, which competed at various ACM Tournaments and World Computer Chess Championships [2] in the 80s and early 90s.




**Proof** Let $\phi^n_j = \phi(x_j, t^n)$ for $\phi\in C^1_0$. Then, 
$$
\begin{aligned}
 0 = & \sum_n\sum_j \Big(\frac{u^{n+1}_j - u^n_j}{\Delta t} + \frac{\hat{f}_{j+1/2}-\hat{f}_{j-1/2}}{\Delta x}\Big)\phi^n_j \Delta x \Delta t \\
   = & -\sum_n\sum_j \Big(\frac{\phi^{n}_j - \phi^{n-1}_j}{\Delta t}u^n_j + \frac{{\phi}^n_j-\phi_{j-1}^n}{\Delta x} \hat{f}_{j+1/2}\Big) \Delta x \Delta t \\
   \xrightarrow{DCT, conservativity} \int_0^\infty \int^\infty_{-\infty} dxdt = 0
\end{aligned}
$$

We used partial summation
$$
\sum_{j=j_1}^{j_2} a_j(b_j-b_{j-1}) = - \sum_{j=j_1}^{j_2}(a_{j+1}-a_j)b_j - a_{j_1}b_{j_1-1} + a_{j_2}b_{j_2}.
$$

# Examples of Conservative Schemes

